# Examples of all decoders (except Kalman Filter and Naive Bayes)

In this example notebook, we:
1. Import the necessary packages
2. Load a data file (spike trains and outputs we are predicting)
3. Preprocess the data for use in all decoders
4. Run all decoders and print the goodness of fit
5. Plot example decoded outputs

## 1. Import Packages

Below, we import both standard packages, and functions from the accompanying .py files

In [32]:
# Import standard packages

import os
import pickle

import numpy as np
from scipy import io, stats
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from skorch import NeuralNetRegressor
from torch import optim
from xgboost import XGBRegressor

from Neural_Decoding.nn import FNN, GRU, LSTM, RNN
from Neural_Decoding.nonlinear import WienerCascade
from Neural_Decoding.preprocessing_funcs import LagMat

## 2. Load Data
The data for this example can be downloaded at this [link](https://www.dropbox.com/sh/n4924ipcfjqc0t6/AACPWjxDKPEzQiXKUUFriFkJa?dl=0&preview=example_data_s1.pickle). It was recorded by Raeed Chowdhury from Lee Miller's lab at Northwestern.


The data that we load is in the format described below. We have another example notebook, "Example_format_data", that may be helpful towards putting the data in this format.

Neural data should be a matrix of size "number of time bins" x "number of neurons", where each entry is the firing rate of a given neuron in a given time bin

The output you are decoding should be a matrix of size "number of time bins" x "number of features you are decoding"

 

In [6]:
# folder='' #ENTER THE FOLDER THAT YOUR DATA IS IN
folder = "Decoding_data"

with open(os.path.join(folder, "example_data_s1.pickle"), "rb") as f:
    neural_data, vels_binned = pickle.load(f, encoding="latin1")  # If using python 3

## 3. Preprocess Data

### 3A. User Inputs
The user can define what time period to use spikes from (with respect to the output).

In [17]:
BIN_BEFORE = 6  # How many bins of neural data prior to the output are used for decoding
BIN_CURRENT = 1  # Whether to use concurrent time bin of neural data
BIN_AFTER = 6  # How many bins of neural data after the output are used for decoding

### 3B. Format Covariates

In [9]:
# Set decoding input
X = neural_data.astype("float32")  # torch requires float32

# Set decoding output
y = vels_binned.astype("float32")

### 3C. Split into training / testing / validation sets
Note that hyperparameters should be determined using a separate validation set. 
Then, the goodness of fit should be be tested on a testing set (separate from the training and validation sets).

#### User Options

In [12]:
TRAIN_SIZE = 0.7
TEST_SIZE = 0.15
VAL_SIZE = 0.15

#### Split Data

In [14]:
# Get validation data
X_train_test, X_val = train_test_split(X, test_size=VAL_SIZE, shuffle=False)
y_train_test, y_val = train_test_split(y, test_size=VAL_SIZE, shuffle=False)

# Get training and test data
X_train, X_test = train_test_split(
    X_train_test, train_size=TRAIN_SIZE / (TRAIN_SIZE + TEST_SIZE), shuffle=False
)
y_train, y_test = train_test_split(
    y_train_test, train_size=TRAIN_SIZE / (TRAIN_SIZE + TEST_SIZE), shuffle=False
)

## 4. Run Decoders
Note that in this example, we are evaluating the model fit on the validation set

### 4A. Wiener Filter (Linear Regression)`

In [18]:
# Declare model
# Format for Wiener Filter, Wiener Cascade, XGBoost, and Dense Neural Network
# Put in "flat" format, so each "neuron / time" is a single feature
pipe = Pipeline(
    [
        ("scaler", StandardScaler()),
        (
            "lagmat",
            LagMat(
                bin_before=BIN_BEFORE,
                bin_current=BIN_CURRENT,
                bin_after=BIN_AFTER,
                flat=True,
            ),
        ),
        ("linear", LinearRegression()),
    ]
)

# Fit model
pipe.fit(X_train, y_train)

# Get predictions
y_val_pred = pipe.predict(X_val)

# Get metric of fit
R2s_wf = r2_score(y_val, y_val_pred, multioutput="raw_values")
print("R2s:", R2s_wf)

R2s: [0.7243494 0.7159852]


### 4B. Wiener Cascade (Linear Nonlinear Model)

In [21]:
# Declare model
# Format for Wiener Filter, Wiener Cascade, XGBoost, and Dense Neural Network
# Put in "flat" format, so each "neuron / time" is a single feature
pipe = Pipeline(
    [
        ("scaler", StandardScaler()),
        (
            "lagmat",
            LagMat(
                bin_before=BIN_BEFORE,
                bin_current=BIN_CURRENT,
                bin_after=BIN_AFTER,
                flat=True,
            ),
        ),
        ("wc", MultiOutputRegressor(WienerCascade(degree=3))),
    ]
)

# Fit model
pipe.fit(X_train, y_train)

# Get predictions
y_val_pred = pipe.predict(X_val)

# Get metric of fit
R2s_wc = r2_score(y_val, y_val_pred, multioutput="raw_values")
print("R2s:", R2s_wc)

R2s: [0.73109653 0.73250392]


### 4C. XGBoost (Extreme Gradient Boosting)

In [25]:
# Declare model
# Format for Wiener Filter, Wiener Cascade, XGBoost, and Dense Neural Network
# Put in "flat" format, so each "neuron / time" is a single feature
pipe = Pipeline(
    [
        ("scaler", StandardScaler()),
        (
            "lagmat",
            LagMat(
                bin_before=BIN_BEFORE,
                bin_current=BIN_CURRENT,
                bin_after=BIN_AFTER,
                flat=True,
            ),
        ),
        ("xgb", XGBRegressor(max_depth=3, n_estimators=200, learning_rate=0.3)),
    ]
)

# Fit model
pipe.fit(X_train, y_train)

# Get predictions
y_val_pred = pipe.predict(X_val)

# Get metric of fit
R2s_xgb = r2_score(y_val, y_val_pred, multioutput="raw_values")
print("R2s:", R2s_xgb)

R2s: [0.75399697 0.76834154]


### 4D. SVR (Support Vector Regression)

In [29]:
# Declare model
# Format for Wiener Filter, Wiener Cascade, XGBoost, and Dense Neural Network
# Put in "flat" format, so each "neuron / time" is a single feature
# The SVR works much better when the y values are normalized, so we first z-score the y values
# We use TransformedTargetRegressor to do that.
pipe = TransformedTargetRegressor(
    regressor=Pipeline(
        [
            ("scaler", StandardScaler()),
            (
                "lagmat",
                LagMat(
                    bin_before=BIN_BEFORE,
                    bin_current=BIN_CURRENT,
                    bin_after=BIN_AFTER,
                    flat=True,
                ),
            ),
            ("svr", MultiOutputRegressor(SVR(max_iter=4000, C=5))),
        ]
    ),
    transformer=StandardScaler(),
)

# Fit model
pipe.fit(X_train, y_train)

# Get predictions
y_val_pred = pipe.predict(X_val)

# Get metric of fit
R2s_svr = r2_score(y_val, y_val_pred, multioutput="raw_values")
print("R2s:", R2s_svr)

/gpfs/laur/personal/woot/mambaforge/envs/neural_decoding/lib/python3.14/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=4000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/gpfs/laur/personal/woot/mambaforge/envs/neural_decoding/lib/python3.14/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=4000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


R2s: [0.8152633  0.82521187]


### 4E. Dense Neural Network

In [31]:
# Declare model
# Format for Wiener Filter, Wiener Cascade, XGBoost, and Dense Neural Network
# Put in "flat" format, so each "neuron / time" is a single feature
pipe = Pipeline(
    [
        ("scaler", StandardScaler()),
        (
            "lagmat",
            LagMat(
                bin_before=BIN_BEFORE,
                bin_current=BIN_CURRENT,
                bin_after=BIN_AFTER,
                flat=True,
            ),
        ),
        (
            "fnn",
            NeuralNetRegressor(
                module=FNN,
                lr=0.001,
                iterator_train__shuffle=True,
                optimizer=optim.Adam,
                batch_size=32,
                module__n_targets=y_train.shape[1],
                module__num_units=400,
                module__frac_dropout=0.25,
                module__n_layers=2,
                max_epochs=10,
                verbose=0,
            ),
        ),
    ]
)

# Fit model
pipe.fit(X_train, y_train)

# Get predictions
y_val_pred = pipe.predict(X_val)

# Get metric of fit
R2s_dnn = r2_score(y_val, y_val_pred, multioutput="raw_values")
print("R2s:", R2s_dnn)

R2s: [0.84426105 0.8625548 ]


### 4F. Simple RNN

In [34]:
# Declare model
# Function to get the covariate matrix that includes spike history from previous bins
# Format for recurrent neural networks (SimpleRNN, GRU, LSTM)
pipe = Pipeline(
    [
        ("scaler", StandardScaler()),
        (
            "lagmat",
            LagMat(
                bin_before=BIN_BEFORE,
                bin_current=BIN_CURRENT,
                bin_after=BIN_AFTER,
                flat=False,
            ),
        ),
        (
            "rnn",
            NeuralNetRegressor(
                module=RNN,
                lr=0.001,
                iterator_train__shuffle=True,
                optimizer=optim.RMSprop,
                batch_size=32,
                module__n_targets=y_train.shape[1],
                module__num_units=400,
                module__frac_dropout=0,
                max_epochs=10,
                verbose=0,
            ),
        ),
    ]
)

# Fit model
pipe.fit(X_train, y_train)

# Get predictions
y_val_pred = pipe.predict(X_val)

# Get metric of fit
R2s_rnn = r2_score(y_val, y_val_pred, multioutput="raw_values")
print("R2s:", R2s_rnn)

R2s: [0.7475089  0.72976804]


### 4G. GRU (Gated Recurrent Unit)

In [35]:
# Declare model
# Function to get the covariate matrix that includes spike history from previous bins
# Format for recurrent neural networks (SimpleRNN, GRU, LSTM)
pipe = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("lagmat", LagMat(bin_before=6, bin_current=1, bin_after=6, flat=False)),
        (
            "gru",
            NeuralNetRegressor(
                module=GRU,
                lr=0.001,
                iterator_train__shuffle=True,
                optimizer=optim.RMSprop,
                batch_size=32,
                module__n_targets=y_train.shape[1],
                module__num_units=400,
                module__frac_dropout=0,
                max_epochs=5,
                verbose=0,
            ),
        ),
    ]
)

# Fit model
pipe.fit(X_train, y_train)

# Get predictions
y_val_pred = pipe.predict(X_val)

# Get metric of fit
R2s_gru = r2_score(y_val, y_val_pred, multioutput="raw_values")
print("R2s:", R2s_gru)

R2s: [0.7291446  0.71711385]


### 4H. LSTM (Long Short Term Memory)

In [36]:
# Declare model
# Function to get the covariate matrix that includes spike history from previous bins
# Format for recurrent neural networks (SimpleRNN, GRU, LSTM)
pipe = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("lagmat", LagMat(bin_before=6, bin_current=1, bin_after=6, flat=False)),
        (
            "lstm",
            NeuralNetRegressor(
                module=LSTM,
                lr=0.001,
                iterator_train__shuffle=False,
                optimizer=optim.RMSprop,
                batch_size=32,
                module__n_targets=y_train.shape[1],
                module__num_units=400,
                module__frac_dropout=0,
                max_epochs=5,
                verbose=0,
            ),
        ),
    ]
)

# Fit model
pipe.fit(X_train, y_train)

# Get predictions
y_val_pred = pipe.predict(X_val)

# Get metric of fit
R2s_lstm = r2_score(y_val, y_val_pred, multioutput="raw_values")
print("R2s:", R2s_lstm)

R2s: [0.75541097 0.7309768 ]
